In [2]:
import pandas as pd
import mysql.connector
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, log_loss, roc_auc_score,f1_score

from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.combine import SMOTETomek
from collections import Counter

%matplotlib inline

In [3]:
# Connect to SQL Server
conn = mysql.connector.connect(host="RGVyavahare", user="root", password="root", database="GooglePlayStore")
# Fetch Data
query = "SELECT * FROM rating_pred"
df = pd.read_sql(query, conn)
conn.close()
df = df.drop('id', axis=1)
# Show first few rows
print(df.head())

C:\Users\Dell 7490\AppData\Local\Temp\ipykernel_16812\25544165.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   Category  Size_in_Mb  Content_Rating  Ad_Supported  In_App_Purchases  \
0       0.0        10.0             0.0             0                 0   
1       1.0         2.9             0.0             1                 0   
2       2.0         3.7             0.0             0                 0   
3       3.0         1.8             0.0             1                 0   
4       1.0         6.2             0.0             0                 0   

   Transformed_Rating  Installs  Free  Rating_Count  Editors_Choice  
0                   0      10.0     1           0.0               0  
1                   4    5000.0     1          64.0               0  
2                   0      50.0     1           0.0               0  
3                   5      10.0     1           5.0               0  
4                   0     100.0     1           0.0               0  


In [4]:
df.shape

(2312683, 10)

In [5]:
df.Transformed_Rating.value_counts()

Transformed_Rating
0    1066841
4     588290
5     423009
3     189288
2      41028
1       4227
Name: count, dtype: int64

In [6]:
zero_rating_rows = df[df["Transformed_Rating"] == 0]
rows_to_drop = zero_rating_rows.sample(n=700000, random_state=42)
# Drop these rows from the original DataFrame
df = df.drop(rows_to_drop.index).reset_index(drop=True)
print("New DataFrame Shape:", df.shape)

New DataFrame Shape: (1612683, 10)


In [7]:
df.Transformed_Rating.value_counts()

Transformed_Rating
4    588290
5    423009
0    366841
3    189288
2     41028
1      4227
Name: count, dtype: int64

In [8]:
# Load dataset
X = df.drop("Transformed_Rating", axis=1)  # Features
Y = df["Transformed_Rating"]  # Target variable

print("🔹 Original Class Distribution:", Counter(Y))
print("New DataFrame Shape:", df.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=7,stratify=Y)
smote_tomek = SMOTETomek(random_state=7)
X_train_resampled, Y_train_resampled = smote_tomek.fit_resample(X_train, Y_train)
print("🔹 After SMOTETomek:", Counter(Y_train_resampled))
 
print(f"X shape: {X_train_resampled.shape}, Y shape: {Y_train_resampled.shape}")

🔹 Original Class Distribution: Counter({4: 588290, 5: 423009, 0: 366841, 3: 189288, 2: 41028, 1: 4227})
New DataFrame Shape: (1612683, 10)
🔹 After SMOTETomek: Counter({0: 411793, 1: 406872, 2: 394347, 3: 382047, 5: 371337, 4: 360302})
X shape: (2326698, 9), Y shape: (2326698,)


In [9]:
df.shape

(1612683, 10)

In [24]:
# Initialize and train Bernoulli Naive Bayes
NB = BernoulliNB(binarize=0.0)
NB.fit(X_train_resampled, Y_train_resampled)

y_pred = NB.predict(X_test)
y_prob = NB.predict_proba(X_test)

nb_ac = accuracy_score(Y_test, y_pred)
print("\nBernoulli Naive Bayes Accuracy:", nb_ac)

f1 = f1_score(Y_test, y_pred, average="weighted")
print("\nF1 Score:", f1)



Bernoulli Naive Bayes Accuracy: 0.47844482797821436

F1 Score: 0.44980934487263924


In [25]:
# Initialize and train the model
hgbc = HistGradientBoostingClassifier(max_iter=100, learning_rate=0.1, random_state=7)
hgbc.fit(X_train_resampled, Y_train_resampled)

y_pred = hgbc.predict(X_test)
y_prob = hgbc.predict_proba(X_test)

hgbc_ac = accuracy_score(Y_test, y_pred)
print("HistGradientBoosting Classifier Accuracy:", hgbc_ac)

f1 = f1_score(Y_test, y_pred, average="weighted")
print("\nF1 Score:", f1)

HistGradientBoosting Classifier Accuracy: 0.5778361116565558

F1 Score: 0.6101447496478094


In [26]:
# Initialize and train RandomForestClassifier
rdf_c = RandomForestClassifier(random_state=7)
rdf_c.fit(X_train_resampled, Y_train_resampled)

rdf_pred = rdf_c.predict(X_test)
rdf_prob = rdf_c.predict_proba(X_test)

rdf_ac = accuracy_score(Y_test, rdf_pred)
print("\nRandomForest Accuracy:", rdf_ac)

f1 = f1_score(Y_test, rdf_pred, average="weighted")
print("\nF1 Score:", f1)


RandomForest Accuracy: 0.5863147342421017

F1 Score: 0.600774744141296


In [ ]:
# Initialize and train DecisionTreeClassifier
dtree_c = DecisionTreeClassifier(random_state=7, criterion='entropy', max_depth=10, min_samples_leaf=2, min_samples_split=5)
dtree_c.fit(X_train_resampled, Y_train_resampled)

dtree_pred = dtree_c.predict(X_test)
dtree_prob = dtree_c.predict_proba(X_test)    

dtree_ac = accuracy_score(Y_test, dtree_pred)
print("\nDecisionTreeClassifier Accuracy:", dtree_ac)

f1 = f1_score(Y_test, dtree_pred, average="weighted")
print("\nF1 Score:", f1)


DecisionTreeClassifier Accuracy: 0.5394508117940079

F1 Score: 0.579644796875134


In [ ]:
# Initialize and train XGBoost Classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=7, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train_resampled, Y_train_resampled)

y_pred = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)

xgb_ac = accuracy_score(Y_test, y_pred)
print("\nXGBoost Classifier Accuracy:", xgb_ac)

f1 = f1_score(Y_test, y_pred, average="weighted")
print("\nF1 Score:", f1)

In [ ]:
# Initialize and train LightGBM Classifier
lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
lgb_model.fit(X_train_resampled, Y_train_resampled)

y_pred = lgb_model.predict(X_test)
y_prob = lgb_model.predict_proba(X_test)

lgb_ac = accuracy_score(Y_test, y_pred)
print("\nLightGBM Classifier Accuracy:", lgb_ac)

f1 = f1_score(Y_test, y_pred, average="weighted")
print("\nF1 Score:", f1)

In [10]:
# Initialize and train CatBoost Classifier
Cat_Boost = CatBoostClassifier(verbose=300, n_estimators=4000, learning_rate=0.3, depth=10, early_stopping_rounds=300)
Cat_Boost.fit(X_train_resampled, Y_train_resampled)
 
cb_ac = Cat_Boost.score(X_train_resampled, Y_train_resampled)
print("\nCatBoost Accuracy:", cb_ac)

y_pred = Cat_Boost.predict(X_test)
y_prob = Cat_Boost.predict_proba(X_test)

f1 = f1_score(Y_test, y_pred, average="weighted")
print("\nF1 Score:", f1)

print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred)) 

cb_log_loss = log_loss(Y_test, y_prob)
print("\nLog Loss:", cb_log_loss)

if len(np.unique(Y_test)) == 2:
    cb_roc_auc = roc_auc_score(Y_test, y_prob[:, 1])
    print("\nROC-AUC Score:", cb_roc_auc)

# Save model
with open("finetuning_final.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)

0:	learn: 1.4394097	total: 2.8s	remaining: 3h 6m 22s
300:	learn: 0.7717729	total: 15m 58s	remaining: 3h 16m 15s
600:	learn: 0.7028656	total: 31m 51s	remaining: 3h 12s
900:	learn: 0.6630604	total: 46m 55s	remaining: 2h 41m 24s
1200:	learn: 0.6364508	total: 1h 2m 14s	remaining: 2h 25m 3s
1500:	learn: 0.6139745	total: 1h 17m 31s	remaining: 2h 9m 4s
1800:	learn: 0.5962052	total: 1h 32m 58s	remaining: 1h 53m 31s
2100:	learn: 0.5806179	total: 1h 48m 29s	remaining: 1h 38m 3s
2400:	learn: 0.5672816	total: 2h 4m 11s	remaining: 1h 22m 42s
2700:	learn: 0.5550418	total: 2h 20m	remaining: 1h 7m 20s
3000:	learn: 0.5442202	total: 2h 35m 59s	remaining: 51m 55s
3300:	learn: 0.5343820	total: 2h 52m 6s	remaining: 36m 26s
3600:	learn: 0.5251068	total: 3h 8m 28s	remaining: 20m 53s
3900:	learn: 0.5166714	total: 3h 24m 56s	remaining: 5m 12s
3999:	learn: 0.5140853	total: 3h 30m 22s	remaining: 0us

CatBoost Accuracy: 0.802705379039308

F1 Score: 0.6429256611775568

Classification Report:
              precisio